# Weekly GitHub Activity Fetcher

This notebook fetches your GitHub activity (commits, pull requests, issues) from the last 7 days, as described in the project requirements.

In [ ]:
# Set Up Environment Variables
import os
from dotenv import load_dotenv

# Load environment variables from .env file if present
load_dotenv()

PERSONAL_GITHUB_TOKEN = os.getenv('PERSONAL_GITHUB_TOKEN')
PERSONAL_GITHUB_USERNAME = os.getenv('PERSONAL_GITHUB_USERNAME')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
NOTION_TOKEN = os.getenv('NOTION_TOKEN')
NOTION_DATABASE_ID = os.getenv('NOTION_DATABASE_ID')

if not PERSONAL_GITHUB_TOKEN:
    raise ValueError("PERSONAL_GITHUB_TOKEN must be set in environment variables.")

if not PERSONAL_GITHUB_USERNAME:
    raise ValueError("PERSONAL_GITHUB_USERNAME must be set in environment variables.")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY must be set in environment variables.")

if not NOTION_TOKEN:
    raise ValueError("NOTION_TOKEN must be set in environment variables.")

if not NOTION_DATABASE_ID:
    raise ValueError("NOTION_DATABASE_ID must be set in environment variables.")

In [ ]:
# Import Required Libraries
import requests
from datetime import datetime, timedelta

In [ ]:
# Define Function to Fetch GitHub Activity

def fetch_github_activity(username, token):
    """
    Fetch commits (push events), pull requests, and issues from the last 7 days for the given user.
    Group by repository, event type, and description.
    """
    headers = {
        "Authorization": f"token {token}",
        "Accept": "application/vnd.github.v3+json"
    }
    since = (datetime.now() - timedelta(days=7)).isoformat() + "Z"
    events_url = f"https://api.github.com/users/{username}/events"
    response = requests.get(events_url, headers=headers)
    response.raise_for_status()
    events = response.json()
    
    events = [event for event in events if event.get("type", "") == "PushEvent" or event.get("type", "") == "PullRequestEvent"]
    
    with open("events.json", "w") as f:
        import json
        json.dump(events, f, indent=2)
    
    activity = []
    for event in events:
        created_at = event.get("created_at", "")
        if created_at < since:
            continue
        repo = event["repo"]["name"]
        type_ = event["type"]
        desc = ""
        body = ""
        if type_ == "PushEvent":
            desc = f"Pushed {len(event['payload']['commits'])} commit(s)"
            body = "\n".join(commit["message"] for commit in event["payload"]["commits"])
        elif type_ == "PullRequestEvent":
            pr = event["payload"]["pull_request"]
            desc = f"PR: {pr['title']} (#{pr['number']})"
            body = pr["body"]
        else:
            continue
        activity.append({
            "repo": repo,
            "type": type_,
            "desc": desc,
            "created_at": created_at,
            "body": body
        })
    return activity

In [ ]:
# Fetch and Display GitHub Activity
activity = fetch_github_activity(PERSONAL_GITHUB_USERNAME, PERSONAL_GITHUB_TOKEN)

if activity:
    print("GitHub Activity (last 7 days):\n")
    for item in activity:
        print(f"[{item['created_at']}] {item['repo']} - {item['type']}: {item['desc']} {item['body']}")
else:
    print("No activity found in the last 7 days.")

## Summarize Weekly GitHub Activity

Aggregate the fetched GitHub activity into a readable summary for AI content generation.

In [ ]:
# Summarize Weekly GitHub Activity

def summarize_activity(activity):
    """
    Create a readable summary from the activity list for AI input.
    """
    if not activity:
        return "No activity found in the last 7 days."
    summary_lines = ["Last week:"]
    for item in activity:
        if item['type'] == 'PushEvent':
            summary_lines.append(f"- {item['desc']} in {item['repo']} with content: {item['body']}")
        elif item['type'] == 'PullRequestEvent':
            summary_lines.append(f"- Merged {item['desc']} in {item['repo']} with content: {item['body']}")
    return "\n".join(summary_lines)

weekly_summary = summarize_activity(activity)
print(len(weekly_summary))
print(weekly_summary)

## Generate Marketing Content with AI

Use an AI model (e.g., OpenAI GPT) to generate 5 unique marketing posts for Mon–Fri based on the weekly summary.

In [ ]:
# Generate Marketing Content with AI
import openai
import json

openai.api_key = OPENAI_API_KEY

def generate_linkedin_post(summary):
    prompt = f"""
You are a social media marketer.
Based on the following weekly GitHub activity summary, list 5 unique topics that I worked on that can be later turned into LinkedIn posts.
Be free to add emojis and very tiny bit of humor if relevant.

Provide the response in a JSON format with the following structure with no markdown. Donot wrap the JSON response in any other text or markdown or code blocks:
- heading: A short title for the post
- body: A detailed explanation of the topic suitable for a LinkedIn post

Summary:
{summary}
"""
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

if openai.api_key:
    marketing_posts = generate_linkedin_post(weekly_summary)
    print(len(marketing_posts))
    marketing_posts = json.loads(marketing_posts)["posts"]
    print(marketing_posts)
else:
    print("OPENAI_API_KEY not set. Please add your OpenAI API key to the environment.")

## Save Generated Posts to Notion Database

Use the Notion API to insert each generated post into your Notion database for review and scheduling.

In [ ]:
# Save Generated Posts to Notion Database
import requests
import json


headers = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

def add_post_to_notion(post, date):
    url = f"https://api.notion.com/v1/pages"
    data = {
        "parent": {"database_id": NOTION_DATABASE_ID},
        "properties": {
            "Title": {"title": [{"text": {"content": post["heading"]}}]},
            "Due Date": {"date": {"start": date}},
            "Status": {"select": {"name": "To Do"}},
            "Type": {"select": {"name": "Marketing"}},
        },
        "children": [
            {
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [{"type": "text","text": {"content": post["body"]}}]
                }
            }
        ]
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        print(f"Post {post["heading"]} added to Notion for {date}.")
    else:
        print(f"Failed to add post for {date}: {response.text}")



# Example usage: assuming marketing_posts is a list of dicts with 'body' and 'heading' keys
from datetime import datetime, timedelta
if NOTION_TOKEN and NOTION_DATABASE_ID:
    today = datetime.today()
    for i, post in enumerate(marketing_posts):
        post_date = (today + timedelta(days=i)).strftime('%Y-%m-%d')
        add_post_to_notion(post, post_date)
else:
    print("NOTION_TOKEN or NOTION_DATABASE_ID not set. Please add them to your environment.")